In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
!export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/nfs/workspaces/virtualenvs/deeplearningnlp/lib/python3.8/site-packages/tensorrt

In [3]:
from keras.layers import Dense, Activation, SimpleRNN
from keras.models import Sequential
import numpy as np

2023-02-27 14:41:13.762613: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-27 14:41:16.561773: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-02-27 14:41:16.561995: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-02-27 14:41:16.562019: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [3]:
hidden_neurons = 50
my_optimizer = "sgd"
batch_size = 60
error_function = "mean_squared_error"
output_nonlinearity = "softmax"
cycles = 5
epochs_per_cycle = 3
context = 3

<IPython.core.display.Javascript object>

In [4]:
def create_tesla_text_from_file(textfile="tesla.txt"):
    clean_text_chunks = []
    with open(textfile, "r", encoding="utf-8") as text:
        for line in text:
            clean_text_chunks.append(line)
            clean_text = ("".join(clean_text_chunks)).lower()
            text_as_list = clean_text.split()
    return text_as_list


text_as_list = create_tesla_text_from_file()

<IPython.core.display.Javascript object>

In [5]:
distinct_words = set(text_as_list)
number_of_words = len(distinct_words)
word2index = dict((w, i) for i, w in enumerate(distinct_words))
index2word = dict((i, w) for i, w in enumerate(distinct_words))

<IPython.core.display.Javascript object>

In [6]:
def create_word_indices_for_text(text_as_list):
    input_words = []
    label_word = []
    for i in range(0, len(text_as_list) - context):
        input_words.append((text_as_list[i : i + context]))
        label_word.append((text_as_list[i + context]))
    return input_words, label_word


input_words, label_word = create_word_indices_for_text(text_as_list)

<IPython.core.display.Javascript object>

In [7]:
input_vectors = np.zeros((len(input_words), context, number_of_words), dtype=np.int16)
vectorized_labels = np.zeros((len(input_words), number_of_words), dtype=np.int16)

<IPython.core.display.Javascript object>

In [8]:
for i, input_w in enumerate(input_words):
    for j, w in enumerate(input_w):
        input_vectors[i, j, word2index[w]] = 1
        vectorized_labels[i, word2index[label_word[i]]] = 1

<IPython.core.display.Javascript object>

In [9]:
model = Sequential()
model.add(
    SimpleRNN(
        hidden_neurons,
        return_sequences=False,
        input_shape=(context, number_of_words),
        unroll=True,
    )
)
model.add(Dense(number_of_words))
model.add(Activation(output_nonlinearity))
model.compile(loss=error_function, optimizer=my_optimizer)

2023-02-22 13:05:40.238234: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-22 13:05:40.290454: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-22 13:05:40.291551: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-22 13:05:40.293931: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorF

<IPython.core.display.Javascript object>

In [34]:
for cycle in range(cycles):
    print(">-<" * 50)
    print(" Cycle: %d" % (cycle + 1))
    model.fit(
        input_vectors, vectorized_labels, batch_size=batch_size, epochs=epochs_per_cycle
    )
    test_index = np.random.randint(len(input_words))
    test_words = input_words[test_index]
    print(
        "Generating test from test index %s with words %s: " % (test_index, test_words)
    )
    input_for_test = np.zeros((1, context, number_of_words))
    for i, w in enumerate(test_words):
        input_for_test[0, i, word2index[w]] = 1
    predictions_all_matrix = model.predict(input_for_test, verbose=0)[0]
    predicted_word = index2word[np.argmax(predictions_all_matrix)]
    print(
        "THE COMPLETE RESULTING SENTENCE IS: %s %s"
        % (" ".join(test_words), predicted_word)
    )
print()  # put more cycles in if what you see here is gibberish

>-<>-<>-<>-<>-<>-<>-<>-<>-<>-<>-<>-<>-<>-<>-<>-<>-<>-<>-<>-<>-<>-<>-<>-<>-<>-<>-<>-<>-<>-<>-<>-<>-<>-<>-<>-<>-<>-<>-<>-<>-<>-<>-<>-<>-<>-<>-<>-<>-<>-<
 Cycle: 1
Epoch 1/3
1/1 [==============================] - 0s 7ms/step - loss: 0.1015
Epoch 2/3
1/1 [==============================] - 0s 6ms/step - loss: 0.1014
Epoch 3/3
1/1 [==============================] - 0s 7ms/step - loss: 0.1014
Generating test from test index 1 with words ['would', 'anyone', 'ever']: 
THE COMPLETE RESULTING SENTENCE IS: would anyone ever besides
>-<>-<>-<>-<>-<>-<>-<>-<>-<>-<>-<>-<>-<>-<>-<>-<>-<>-<>-<>-<>-<>-<>-<>-<>-<>-<>-<>-<>-<>-<>-<>-<>-<>-<>-<>-<>-<>-<>-<>-<>-<>-<>-<>-<>-<>-<>-<>-<>-<>-<
 Cycle: 2
Epoch 1/3
1/1 [==============================] - 0s 9ms/step - loss: 0.1014
Epoch 2/3
1/1 [==============================] - 0s 6ms/step - loss: 0.1014
Epoch 3/3
1/1 [==============================] - 0s 7ms/step - loss: 0.1014
Generating test from test index 2 with words ['anyone', 'ever', 'eat']: 
THE COMPLETE

<IPython.core.display.Javascript object>